# Harmonic potential

In [ ]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as mplot3d
import numpy as np

## Simple harmonic oscillators

The potential energy of a harmonic oscillator is $q^2/2$. The force is then $F = -q$, and the Hamiltonian is $H(q, p) = T + U = q^2/2 + p^2/2$.

In [ ]:
def hamiltonian(q, p):
    return np.dot(q, q)/2 + np.dot(p, p)/2

In [ ]:
qq = np.linspace(-1, 1, 10)
pp = np.linspace(-1, 1, 10)

Q, P = np.meshgrid(qq, pp)

E = [[None for _ in range(10)] for _ in range(10)]

for i, p in enumerate(pp):
    for j, q in enumerate(qq):
        E[j][i] = hamiltonian(q, p)
        pass
    pass

In [ ]:
plt.contourf(Q, P, E, 100)
plt.title('Contour of Hamiltonian')
plt.show()

### Equations of motion

The equations of motion are $\dot{q} = p$ and $\dot{p} = -q$.

### RK4 integration

In [ ]:
# RK4 approximation.
q_init = np.array([1, 0])
p_init = np.array([-1, 2])

# time series.
tt = np.linspace(0, 100, 1000)

# time data.
n = len(tt)
t_min, t_max = tt[0], tt[n-1]
h = (t_max - t_min) / (n - 1)

# solve for solution.
qq = [None for _ in range(n)]
pp = [None for _ in range(n)]

# compute energies.
energy = [None for _ in range(n)]

for i in range(n):
    if i == 0:
        qq[0] = q_init
        pp[0] = p_init
        energy[0] = hamiltonian(q_init, p_init)
        pass
    else:
        q, p = qq[i-1], pp[i-1]
        
        # compute k1.
        k1q = h*p
        k1p = -h*q
        
        # compute k2.
        k2q = h*(p + k1q/2)
        k2p = -h*(q + k1p/2)
        
        # compute k3.
        k3q = h*(p + k2q/2)
        k3p = -h*(q + k2p/2)
        
        # compute k4.
        k4q = h*(p + k3q)
        k4p = - h*(q + k3p)
        
        # iterate.
        qq[i], pp[i] = q + 1/6*(k1q + 2*k2q + 2*k3q + k4q), p + 1/6*(k1p + 2*k2p + 2*k3p + k4p)
        
        # compute energy.
        energy[i] = hamiltonian(q, p)
        pass
    pass

In [ ]:
xx = [q[0] for q in qq]
yy = [q[1] for q in qq]

f, axs = plt.subplots(3, 1, figsize=(6.4 * 1, 4.8 * 3))

plt.subplot(3, 1, 1)
plt.plot(tt, xx)
plt.plot(tt, yy)
plt.title('Position evolution graph')

plt.subplot(3, 1, 2)
plt.plot(xx, yy)
plt.title('Solution space graph')

plt.subplot(3, 1, 3)
plt.plot(tt, energy)
plt.title('Energy evolution graph')
plt.show()

In the RK4 case, energy grows exponentially, and mechanical energy is not conserved.

In [ ]:
print('Standard deviation: {}'.format(np.std(energy)))

### Verlet integration

In [ ]:
# solve.
qq = [None for _ in range(n)]
pp = [None for _ in range(n)]
energy = [None for _ in range(n)]

for i in range(n):
    if i == 0:
        qq[0] = q_init
        pp[0] = p_init
        energy[0] = hamiltonian(q_init, p_init)
        pass
    else:
        q, p = qq[i-1], pp[i-1]
        
        _p = p - h*q/2
        q_new = q + h*_p
        acceleration = -q_new
        p_new = _p + h*acceleration/2
        
        qq[i], pp[i] = q_new, p_new
        energy[i] = hamiltonian (q_new, p_new)
        pass
    pass

In [ ]:
xx = [q[0] for q in qq]
yy = [q[1] for q in qq]

f, axs = plt.subplots(3, 1, figsize=(6.4 * 1, 4.8 * 3))

plt.subplot(3, 1, 1)
plt.plot(tt, xx)
plt.plot(tt, yy)
plt.title('Position evolution graph')

plt.subplot(3, 1, 2)
plt.plot(xx, yy)
plt.title('Solution space graph')

plt.subplot(3, 1, 3)
plt.plot(tt, energy)
plt.title('Energy evolution graph')
plt.show()

In [ ]:
print('Standard deviation: {}'.format(np.std(energy)))

In the Verlet case, energy is bounded between $0.498$ and $0.5$. Verlet integration conserves the system's mechanical energy.

## Numerical approximation methods